In [ ]:
# -*- coding: utf-8 -*-
"""CriticaFilmesSeries.ipynb.py
Script Python para dar Sinopse, Crítica e nota de Filmes e Séries com Google Gemini.
"""

# Instalar SDK do Google se necessário
# !pip install -q -U google-generativeai
# !pip install beautifulsoup4 requests

# Importar bibliotecas
import google.generativeai as genai
from google.colab import userdata
import requests
from bs4 import BeautifulSoup

# Configurar API Key (substitua pelo seu valor)
GOOGLE_API_KEY = "GOOGLE_API_KEYE"
genai.configure(api_key=GOOGLE_API_KEY)

# Configurar modelo Gemini
generation_config = {
    "candidate_count": 1,
    "temperature": 0.7,  # Ajustar a criatividade do modelo
}
safety_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE",
}
model = genai.GenerativeModel(
    model_name="gemini-pro",
    generation_config=generation_config,
    safety_settings=safety_settings,
)

def extrair_notas_sites(filme_serie):
    """
    Extrai notas de críticos e público de sites de cinema.
    (Implementação simplificada - Adaptar para diferentes sites)
    """
    notas_criticos = {}
    notas_publico = {}

    # --- ADAPTAR ESTA SEÇÃO PARA SITES REAIS ---
    sites_exemplo = {
        "AdoroCinema": {"criticos": 4/5, "publico": 4.5/5},
        "Omelete": {"criticos": 3.5/5, "publico": 4/5},
        "IGN Brasil": {"criticos": 8/10, "publico": None},
        "Metacritic": {"criticos": 72/100, "publico": None},
        "Rotten Tomatoes": {"criticos": 86, "publico": None}, # 86%
        "IMDb": {"criticos": None, "publico": 8.2/10},
    }

    for site, notas in sites_exemplo.items():
        if notas["criticos"]:
            notas_criticos[site] = notas["criticos"]
        if notas["publico"]:
            notas_publico[site] = notas["publico"]
    # --- FIM DA ADAPTAÇÃO ---

    return notas_criticos, notas_publico

def pesquisar_filme_serie(filme_serie):
    """
    Pesquisa um Filme ou Série usando o Gemini e retorna informações relevantes.
    """
    prompt = f"""
    Você é um crítico de cinema experiente.
    Forneça a sinopse, uma crítica detalhada e uma nota de 0 a 10 para "{filme_serie}".
    """
    response = model.generate_content(prompt)
    critica_gemini = response.text

    notas_criticos, notas_publico = extrair_notas_sites(filme_serie)

    # --- Análise da crítica do Gemini (simplificada) ---
    aspectos_positivos = []
    pontos_relevantes = []

    if "excelente" in critica_gemini.lower():
        aspectos_positivos.append("atuação impecável")
    if "emocionante" in critica_gemini.lower():
        aspectos_positivos.append("enredo envolvente")
    if "fracas" in critica_gemini.lower():
        pontos_relevantes.append("algumas falhas no roteiro")
    if "negativa" in critica_gemini.lower():
        pontos_relevantes.append("recepção negativa do público")
    # ... adicione mais lógica para identificar outros aspectos

    # --- Fim da análise ---

    # Formatar saída das notas
    saida_criticos = "\nMédia das notas de críticos especialistas:\n"
    for site, nota in notas_criticos.items():
        if isinstance(nota, float) and nota <= 1:  # Se a nota for decimal e menor ou igual a 1
            nota_formatada = f"{int(nota * 10)}/10"
        elif isinstance(nota, float):
            nota_formatada = f"{nota:.1f}/5"
        else:
            nota_formatada = f"{nota}%"
        saida_criticos += f"  • {site}: {nota_formatada}\n"

    saida_publico = "\nMédia das notas do público:\n"
    for site, nota in notas_publico.items():
        if isinstance(nota, float) and nota <= 1:
            nota_formatada = f"{int(nota * 10)}/10"
        elif isinstance(nota, float):
            nota_formatada = f"{nota:.1f}/5"
        else:
            nota_formatada = f"{nota}%"
        saida_publico += f"  • {site}: {nota_formatada}\n"

    # Conclusão (utilizando os aspectos extraídos)
    conclusao = "\nConclusão:\n"
    if aspectos_positivos:
        conclusao += f"O filme/série recebeu críticas geralmente positivas, com destaques para {', '.join(aspectos_positivos)}. "
    else:
        conclusao += "O filme/série recebeu críticas mistas, "
    if pontos_relevantes:
        conclusao += f"e a recepção do público apresentou alguns pontos a serem considerados, como {', '.join(pontos_relevantes)}."

    resultado = f"""
    --- Resultados da Pesquisa para {filme_serie} ---

    {critica_gemini}

    {saida_criticos}
    {saida_publico}

    {conclusao}

    -------------------------------------------------
    """
    return resultado

# Loop principal
while True:
    tema = input("Digite um nome de um Filme ou Série (ou 'sair' para encerrar): ")
    if tema.lower() == "sair":
        break
    resultado = pesquisar_filme_serie(tema)
    print(resultado)